# Including Single Particle and First Band Approximation Functions

In [2]:
using NBInclude
@nbinclude("Kagome SP.ipynb")
@nbinclude(joinpath("..","First Band Approximation Functions.ipynb"))

get_num_mb_op

# Initial Parameters

In [3]:
Nx = 5; Ny = 5; N=Nx*Ny*3; cut_off = 5;
PN = [0,1,2];
U = 2;
t1 = -1; L1 = 0.28; t2 = 0.3; L2 = 0.2;
# t1=-1;L1=0 ;t2=L2=0 # t1=L1=0;t2=L2=-1 # t1=t2=-1;L1=L2=0

# Single Particle

In [4]:
sp_basis = NLevelBasis(N)
sp_matrix = get_SP_H(Nx, Ny, t1, L1, t2, L2);

In [5]:
H1 = get_sp_op_data(sp_basis, N, sp_matrix);

In [6]:
#check operator form
eigenenergies(dense(H1)) == eigenenergies(dense(sp_matrix))

true

In [7]:
sub_states = get_sub_states(H1, cut_off);

In [8]:
basis_sub, P, Pt = get_projector_op(sub_states, sp_basis);

In [9]:
H1_sub = get_subspace_op(H1, P, Pt);

In [10]:
num_sub_list = get_num_sub_list(N, sp_basis, P, Pt);

# Many Body

In [11]:
states_mb = bosonstates(basis_sub, PN) 
basis_mb = ManyBodyBasis(basis_sub, states_mb);

In [12]:
H1_MB = get_mb_op(basis_mb, basis_sub, H1_sub);

In [13]:
@nbinclude("Kagome MB .ipynb"; regex=r"#.*executeme");

In [14]:
basis_cut_mb, basis_cut_sp = get_Bosonic_MB_Basis(cut_off,PN)
H_Int = Hubbard_Interaction(basis_cut_sp, basis_cut_mb, P, Pt, cut_off);

# Constructing Total Hamiltonian

In [70]:
H1cut = SparseOperator(basis_cut_mb)
H1cut.data = H1_MB.data

number_mb_list_operators = get_num_mb_op(N, basis_cut_sp, num_sub_list, basis_cut_mb, basis_sub)

V = 4;V_imp_site=10
V_imp = V * number_mb_list_operators[V_imp_site];
H_total = H1cut + H_Int + V_imp;

# Exact Diagonalization

In [71]:
using DataFrames

E, UU = eigenstates(dense(dense((H_total+dagger(H_total))/2)))

function get_energies(pn, E, UU, basis)
    PN_Energies = Array{Float64}(undef, length(E), 2)
    for i in 1:length(E)
        PN_Energies[i] = round(expect(number(basis), UU[i])) #expected values (first column)
        PN_Energies[i,2] = E[i] #eigen-values (second column)
    end
    
    # filter
    df = DataFrame(PN_Energies, :auto)
    df = filter(row -> (row.x1 == pn),  df)
    
    return df
end

get_energies (generic function with 1 method)

In [72]:
pn = 2.0
df = get_energies(pn, E, UU, basis_cut_mb)
filtered_energies = Matrix(df)[:,2]
show(stdout,"text/plain", filtered_energies) #show all outputs

15-element Vector{Float64}:
 -5.564139907528857
 -5.524054213929802
 -5.520793892606464
 -5.51869157795927
 -5.515221205759301
 -5.505853345298276
 -5.504237679834005
 -5.501468409532167
 -5.497498899770877
 -5.476089282490228
 -5.199485646644139
 -5.184948665684033
 -5.177562944015835
 -5.17102475569663
 -4.834682876414361

Allta ki fonksiyonun çalışması için, dizide ki filtre edilmiş parçacık sayısı her zaman en büyük değer de olmalıdır.
Örneğin, `PN=[0,1,2,3,4]` iken filtre edilen parçacık sayısı `pn=4` olmalıdır!

In [73]:
function Restricted_Hubbard_States(states)
    number_of_states = length(filtered_energies)
    return states[1:number_of_states];
end

Restricted_Hubbard_States (generic function with 1 method)

In [74]:
r_hubbard_states = Restricted_Hubbard_States(UU);

# Density-Profile

In [75]:
# METHOD 1

NM_MB_Array_Storage = zeros(Complex{Float64},length(basis_cut_mb),length(basis_cut_mb),N);
NM_MB_Matrix = zeros(Complex{Float64},length(basis_cut_mb),length(basis_cut_mb));
for m in 1:N
    for i in 1:length(basis_cut_mb)
        for j in 1:length(basis_cut_mb)
            #NM_MB_Matrix[i,j] = num_mb_list[m].data[i,j]
            NM_MB_Matrix[i,j] = number_mb_list_operators[m].data[i,j]
            # NOT: num_mb_list ile number_mb_list_operators bazları farklı ama içerikleri aynı!
        end
    end
    NM_MB_Array_Storage[:,:,m] = NM_MB_Matrix
end

BL = BR = basis_cut_mb
index_number_op = 5
T = NM_MB_Array_Storage[:,:,index_number_op]
Op = Operator(BL,BR,T)
index_eigen_states = 1
expect(Op, r_hubbard_states[index_eigen_states])

# METHOD 2
# expect(number_mb_list_operators[index_number_op],r_hubbard_states[index_eigen_states])

0.021893961199270694 + 3.8624702394796095e-19im

In [76]:
Sum = 0
total_expect_list=[]

for i in 1:N
    T = NM_MB_Array_Storage[:,:,i]
    Op = Operator(BL,BR,T)
    n_i = 0
    for index_eig_states in 1:length(filtered_energies)       
        Sum += expect(Op, r_hubbard_states[index_eig_states])
        #println(i,"\t",expect(Op, r_hubbard_states[index_eig_states]))
        n_i += expect(Op, r_hubbard_states[index_eig_states])
    end
    push!(total_expect_list,n_i)
end

In [77]:
index_eig_states = 1
Sum = 0
expect_list=[]

for i in 1:N
    T = NM_MB_Array_Storage[:,:,i]
    Op = Operator(BL,BR,T)      
    Sum += expect(Op, r_hubbard_states[index_eig_states])
    #println(i,"\t",expect(Op, r_hubbard_states[index_eig_states]))
    push!(expect_list,expect(Op, r_hubbard_states[index_eig_states]))
end
Sum

1.9999999999999991 + 5.07483908479653e-18im

In [78]:
real(expect_list)

75-element Vector{Float64}:
 0.032335135399728286
 0.027990807852828644
 0.02826247571240156
 0.032935430022932796
 0.021893961199270694
 0.043813059089678086
 0.03633846265437041
 0.042467428165629616
 0.0151956626298603
 0.000585258079395527
 0.015639928478228187
 0.01394232514228701
 0.03607716414548214
 ⋮
 0.028134760339955732
 0.028027704323732357
 0.011037412758724477
 0.008490232372924499
 0.022299282299522716
 0.019633440308118134
 0.04369465167858269
 0.017472368682704797
 0.058958878807536884
 0.049312380711515336
 0.048232817456938484
 0.026106979435380942

---------------

In [79]:
# num_sub_list = get_num_sub_list(sp_basis,P,Pt);

In [80]:
# num_mb_list = get_num_mb_list(basis_mb, b_sub, num_sub_list);

## num_mb_list[i] & number(bcut_mb,i)

In [81]:
# num_mb_list[7] #sub space'e project edilmiş number operator!
# number(bcut_mb,7); #sub space'e project edilmemiş number operator!

## Error (IT DOESN'T WORK?)

In [82]:
# typeof(dense(num_mb_list[1]))

In [83]:
# typeof(S[1])

In [84]:
# expect(num_mb_list[1],S[1])

In [85]:
# Find x and y coordinates from given site index

function exp_list0(site_indx)
    
    x_co = OffsetArray(get_sites(Nx, Ny, a1_vec, a2_vec, Basis)[4], 1:Nx*Ny*3)
    y_co = OffsetArray(get_sites(Nx, Ny, a1_vec, a2_vec, Basis)[5], 1:Nx*Ny*3)
    
    x = hcat(x_co, y_co)[site_indx, 1]
    y = hcat(x_co, y_co)[site_indx, 2] 
    exp_val = real(expect_list)[site_indx] 
    
    return x, y, exp_val
end
exp_list0(1)

(0.0, 0.0, 0.032335135399728286)

In [86]:
# Find site_index from given x and y coordinates

function exp_list1(Xx, Yy)
    co_list = hcat(x_co, y_co)
    site_indx = intersect(findall(x->x==Xx, co_list[:,1]), findall(x->x==Yy, co_list[:,2]))
    return real(expect_list)[site_indx] 
end

x_co = OffsetArray(get_sites(Nx, Ny, a1_vec, a2_vec, Basis)[4], 1:Nx*Ny*3)
y_co = OffsetArray(get_sites(Nx, Ny, a1_vec, a2_vec, Basis)[5], 1:Nx*Ny*3)
site_index = 2
Xx = x_co[site_index]
Yy = y_co[site_index]
print(site_index," => ",Xx,"\t",Yy," => ",exp_list1(Xx, Yy))

2 => 1.0	0.0 => [0.027990807852828644]

In [88]:
x = Float64[]
for i in x_co
    push!(x,i)
end
y = Float64[]
for i in y_co
    push!(y,i)
end
z = exp_list1.(x, y)
z = collect(Iterators.flatten(z))
surface(x,y,z,xlabel="x",ylabel="y", camera=(0,90))
title!("t1=$(t1),L1=$(L1),t2=$(t2),L2=$(L2) \n Nx=$(Nx), Ny=$(Ny), N=$(N), Cut-off=$(cut_off) \n PN=$(pn), U=$(U), V_imp=$(V) \n V_imp_site=$(V_imp_site)")
scatter!(x,y,0 .* z,camera=(0,90), legend=false)
savefig("V_imp_site=10.png") # Plot and then save it!

"C:\\Users\\Can\\Downloads\\GITHUB FOLDER\\Kagome-Lattice\\My Numerical Calculations\\Many-Body\\Kagome\\V_imp_site=10.png"

# Comparansion

In [ ]:
#U çok küçükken (Band-Gap büyüdüğünde) subspace enerjileri, real-space enerjilerine yakınsar:

MB_Hamiltonian = Kagome_Finite_U(Nx,Ny,Basis_MB,U,sp_op)
real_space_finite_u = eigenenergies(dense(MB_Hamiltonian)); sub_space_finite_u = E_States[1]
plot(1:length(real_space_finite_u[1:21]),real_space_finite_u,seriestype=:scatter,markershape=:star5,markersize=7,label="Real-Space")
plot!(1:length(sub_space_finite_u),sub_space_finite_u,seriestype=:scatter,label="Sub-Space",legend=:topleft)
title!("U=$(U)");xlabel!("n");ylabel!("E")